In [ ]:
import pandas as pd

# Data sources:

https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv

https://www.kaggle.com/datasets/shuyangli94/foodcom-recipes-with-search-terms-and-tags

https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews

# Read data

Download all data sources and unpack then in src directory

In [ ]:
# read cvs file "PP_recipes.csv" and print first 10 rows
raw_recipes = pd.read_csv('src/RAW_recipes.csv')
raw_recipes = raw_recipes[['id', 'minutes', 'submitted', 'tags']]
raw_recipes.head(2)

In [ ]:
recipes_with_ingredients = pd.read_csv('src/recipes_w_search_terms.csv')
recipes_with_ingredients = recipes_with_ingredients[['id', 'name', 'ingredients_raw_str', 'ingredients', 'steps', 'description']]
recipes_with_ingredients.head(2)

In [ ]:
recipes_big = pd.read_parquet('src/recipes_with_imgs.parquet', engine='fastparquet')
print(recipes_big.columns)
recipes_big = recipes_big[['RecipeId', 'AuthorId', 'AuthorName', 'ReviewCount']]
recipes_big.head(2)

In [ ]:
recipes_tokenized = pd.read_csv('src/PP_recipes.csv')
recipes_tokenized = recipes_tokenized[['id', 'calorie_level', 'ingredient_ids']]
recipes_tokenized.head(2)

# Prepare recipes base

In [ ]:
# merge interactions_count with recipes based in id
recipes_full = pd.merge(raw_recipes, recipes_with_ingredients, left_on='id', right_on='id', how='left')
# merge interactions_count with recipes based in id
recipes_full = pd.merge(recipes_full, recipes_tokenized, left_on='id', right_on='id', how='left')
# merge with recipes_big
recipes_full = pd.merge(recipes_full, recipes_big, left_on='id', right_on='RecipeId', how='left')

recipes_full = recipes_full.dropna(subset=['AuthorId', 'AuthorName'])

In [ ]:
# sort recipes by review count and take top 1000
base_recipes = recipes_full.sort_values(by=['ReviewCount'], ascending=False)
base_recipes.dropna()

# drop recipes where ingredient_ids are nan
base_recipes = base_recipes.dropna(subset=['ingredient_ids'])
base_recipes = base_recipes.head(5000)


base_recipes

# Ingredients table

In [ ]:
ingredients = pd.read_pickle('src/ingr_map.pkl')

# create new ingredients table data, that stores only processed and id columns off current ingredients data
ingredients_processed = ingredients[['id', 'replaced']]
ingredients_processed.columns = ['id', 'name']

# remove repetitions in ingredients_processed
ingredients_processed = ingredients_processed.drop_duplicates()
# save processed ingredients to csv file
ingredients_processed.to_csv('result/ingredients.csv', index=False)

ingredients_processed

# Recipes-ingredients table

In [ ]:
import ast
# create empty pandas dataframe
recipes_ingredients = pd.DataFrame()

to_drop = []
for index, row in base_recipes.iterrows():
    try:
        ingredients_raw = ast.literal_eval(row['ingredients_raw_str'])
        ingredients_ids = ast.literal_eval(row['ingredient_ids'])
        ingredients_names = ast.literal_eval(row['ingredients'])
    except:
        to_drop.append(index)
        continue

    if len(ingredients_names) > len(ingredients_ids):
        ingredients_names = ingredients_names[:len(ingredients_ids)]
    elif len(ingredients_names) < len(ingredients_ids):
        ingredients_ids = ingredients_ids[:len(ingredients_names)]

    if len(ingredients_raw) > len(ingredients_names):
        ingredients_raw = ingredients_raw[:len(ingredients_names)]
    elif len(ingredients_raw) < len(ingredients_names):
        ingredients_names = ingredients_names[:len(ingredients_raw)]
    
    ingredients_quantity = []
    for ingredient_raw, ingredient_name in zip(ingredients_raw, ingredients_names):
        if ingredient_name in ingredient_raw:
            quantity = ingredient_raw.split(ingredient_name)[0].strip()
            ingredients_quantity.append(quantity)
        else:
            ingredients_quantity.append('')
    # create dataframe with names and id's 
    ingredients_array = pd.DataFrame({'ingredient_id': ingredients_ids, 'quantity': ingredients_quantity, })

    # add row id column to tags_array
    ingredients_array['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_ingredients = pd.concat([recipes_ingredients, ingredients_array], ignore_index=True)

# drop recipes with index in to_drop
base_recipes = base_recipes.drop(to_drop)

# change the order of columns in recipes_ingredients
recipes_ingredients = recipes_ingredients[['recipe_id', 'ingredient_id', 'quantity' ]]

# drop duplicate rows, where recipe_id and ingredient_id are the same
recipes_ingredients = recipes_ingredients.drop_duplicates(subset=['recipe_id', 'ingredient_id'])

recipes_ingredients.to_csv('result/recipes_ingredients.csv', index=False)
print(len(to_drop), len(base_recipes))
recipes_ingredients.head(2)

# Tags table

In [ ]:
forbidden_tags = ['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 
'north-american', 'equipment', '4-hours-or-less', '5-ingredients-or-less', 'beef', 'crock-pot-slow-cooker',
 'roast-beef', 'taste-mood', 'poultry', 'chicken', 'chicken-breasts', 'corn', 'southern-united-states',
  'bananas', 'holiday-event', 'southwestern-united-states', 'apples', 'beans', 'eggs', 'beef-ribs',
   'black-beans', 'brown-rice', 'refrigerator', 'tuna', 'grapes', 'tomatoes', 'salmon', 'pork']

In [ ]:
import ast
# create empty pandas dataframe
tags = pd.DataFrame()

# iterate for every row in recipes_with_interactions_1
for index, row in base_recipes.iterrows():
    tags_array = ast.literal_eval(row['tags'])
    tags_array = filter(lambda x: x not in forbidden_tags, tags_array)
    # add tags to tags dataframe
    tags = pd.concat([tags, pd.DataFrame(tags_array)], ignore_index=True)

# remove repetitions in tags
tags = tags.drop_duplicates()
# name first column in tags "tag_name"
tags.columns = ['name']
tags['id'] = tags.index

tags = tags[['id', 'name']]
# add id column to tags

tags.to_csv('result/tags.csv', index=False)
tags.head(2)

# Recipes-tags table

In [ ]:
# create empty pandas dataframe
recipes_tags = pd.DataFrame()

# iterate for every row in recipes_with_interactions_1
for index, row in base_recipes.iterrows():
    tags_array = ast.literal_eval(row['tags'])
    tags_array = filter(lambda x: x not in forbidden_tags, tags_array)

    # find tag id for every tag name in tags_array
    tags_ids = tags[tags['name'].isin(tags_array)]['id'].values

    tags_ids_df = pd.DataFrame(tags_ids, columns=['tag_id'])

    # add row id column to tags_array
    tags_ids_df['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_tags = pd.concat([recipes_tags, tags_ids_df], ignore_index=True)

# change the order of columns in recipes_tags
recipes_tags = recipes_tags[['recipe_id', 'tag_id']]
recipes_tags.to_csv('result/recipes_tags.csv', index=False)

recipes_tags.head(2)

# Recipes-steps table

In [ ]:
recipes_steps = pd.DataFrame()

for index, row in base_recipes.iterrows():
    steps = ast.literal_eval(row['steps'])
    
    # create dataframe with names and id's 
    steps = pd.DataFrame({'step_content': steps, 'step_number': range(1, len(steps) + 1)})

    # add row id column to tags_array
    steps['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_steps = pd.concat([recipes_steps, steps], ignore_index=True)

# change the order of columns in recipes_ingredients
recipes_steps = recipes_steps[['recipe_id', 'step_number', 'step_content']]

recipes_steps.to_csv('result/recipes_steps.csv', index=False)
recipes_steps

# Users table

In [ ]:
from passlib.context import CryptContext
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# find distinct contributor_id in recipes_table
users = base_recipes[['AuthorId', 'AuthorName']].drop_duplicates()

# change the name of the first column "id"
users.columns = ['id', 'username']
users['id'] = users['id'].astype(int)

users['username'] = users['username'].str.replace(" ", "")
# add email column which is equal to id
users['email'] = users['username'].str.lower() + '@unknown.com'
users['hashed_password'] =  pwd_context.hash('password123')
users['last_login'] =  '2023-01-20'

users.to_csv('result/users.csv', index=False)
users

# Recipes table

In [ ]:
recipes_table = base_recipes.drop(['steps', 'ingredients', 'ingredient_ids', 'ingredients_raw_str', 'tags', 'RecipeId', 'AuthorName'], axis=1)

# change column name call ReviewCount to reactions
recipes_table = recipes_table.rename(columns={'ReviewCount': 'likes', 'submitted': 'date_added', 'name': 'title', 'AuthorId': 'contributor_id'})
recipes_table = recipes_table[['id', 'contributor_id', 'title', 'minutes', 'description',  'date_added', 'likes', 'calorie_level']]

recipes_table['description'] = recipes_table['description'].str.capitalize()
recipes_table['contributor_id'] = recipes_table['contributor_id'].astype(int)
recipes_table['calorie_level'] = recipes_table['calorie_level'].astype(int)
recipes_table['likes'] = recipes_table['likes'].astype(int)

recipes_table.to_csv('result/recipes.csv', index=False)

recipes_table